# This is the pcdet-free version of beamng 3d detection

# Pcdet解耦版本的beamng 3d目标检测

## Install-安装

- create environment and activate it in anaconda-创建并激活anaconda环境

  ```bash
  conda env create -f ./environment.yaml
  conda activate env_beampy_torch
  ```

- install spconv from https://github.com/traveller59/spconv 安装spconv

  for example 例如:

  depends on your version of cuda 注意取决于你的cuda版本

  ```
  pip install spconv-cu118
  ```

## Launch-启动

Run all the code cell in order while following the instructions 按照步骤每个代码块逐个启动即可

In [ ]:
from beamng_control import beamng_server, beamng_client
from detectors import torch_script_module
from utils import *
from utils.mods.post_processing import mono_label_distance_tracker
import open3d as o3d
import numpy as np
import torch

import time
from IPython.display import clear_output, display

In [ ]:
logger = create_logger()
LAUNCH_SERVER = True
REMOTE_IP = "192.168.31.13" # only for server == False

In [ ]:
if LAUNCH_SERVER:
    server_a = beamng_server() 
client_b = beamng_client(host="127.0.0.1" if LAUNCH_SERVER else REMOTE_IP, logger=logger)

In [ ]:
if LAUNCH_SERVER:
    server_a.init_server()
    server_a.launch_server()

In [ ]:
client_b.init_client()
client_b.launch_client()

In [ ]:
if LAUNCH_SERVER:
    server_a.debug_luanch_test()

In [ ]:
CFG_FILE = ".\\configs\\kitti_prepocessing.yaml"

cfg = cfg_from_yaml_file(CFG_FILE, cfg)
cfg.DATA_CONFIG.DATA_PROCESSOR

In [ ]:
client_b.debug_luanch_test(None, lidar_para={
    "requested_update_time": 0.01, 
    "is_visualised": True,
    "is_using_shared_memory": True,
    "vertical_resolution": 64,
    # "horizontal_angle": 90,
    # "dir":(-0.3, -0.5, 0), Not work.
})
pcs_dataset = beamng_point_cloud_dataset(dataset_cfg = cfg.DATA_CONFIG, logger=logger, lidar=client_b.lidar_t, class_names=cfg.CLASS_NAMES)
# model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=pcs_dataset)
# model.load_params_from_file(filename=CKPT_FILE, logger=logger, to_cpu=True)
object_tracker = mono_label_distance_tracker(track_length=10)
model = torch.jit.load("./torch_scripts/point_pillar_model.pt")
# model = torch_script_module(path="./torch_scripts/point_pillar_model.pt")
model.cuda()
model.eval()

In [ ]:
vis = o3d.visualization.Visualizer()
vis.create_window()

with torch.no_grad():
    for idx, data_dict in enumerate(pcs_dataset):
        pre_time = data_dict['pre_time']
        before_time = time.perf_counter()
        
        data_dict = pcs_dataset.collate_batch([data_dict])
        load_data_to_gpu(data_dict)

        pred_dicts, _ = model.forward(data_dict)
        
        cls_preds = pred_dicts["pred_scores"]
        box_preds = pred_dicts["pred_boxes"]
        label_preds = pred_dicts["pred_labels"]

        selected, selected_scores = class_agnostic_nms(
                            box_scores=cls_preds, box_preds=box_preds,
                            score_thresh=0.4
                        )

        final_scores = selected_scores
        final_labels = label_preds[selected]
        final_boxes = box_preds[selected]
        
            
        # if final_boxes is not None:
        #     object_tracker.updates_object(final_boxes[np.newaxis, :, :])

        # box = object_tracker.get_all_bounding_box()
        # tracks = object_tracker.get_all_tracks()

        draw_scenes(vis,
            points=data_dict['points'][:, 1:], ref_boxes=final_boxes,
            ref_scores=final_scores, ref_labels=final_labels, confidence=None, tracks=None
        )
        
        # draw_scenes(vis,
        #     points=data_dict['points'][:, 1:], ref_boxes=None,
        #     ref_scores=None, ref_labels=None, confidence=None, tracks=None
        # )
        
        
        
        after_time = time.perf_counter()
        
        
        # clear_output(wait=True)
        # logger.info(f"Compute time: {after_time - before_time:.3f} + {pre_time:.3f} == {after_time - before_time + pre_time:.3f}s")
        # logger.info(f"Target amount: {len(final_boxes)}")
        # logger.info(f"Current uuid: {object_tracker.get_last_uuid()}")
        
        # tensorized_final_boxes = torch.stack(final_boxes)[:, 0:3].view(1, -1, 3)
        # tesnorized_final_labels = torch.stack(final_labels).view(1, -1)

        # draw_scenes(vis,
        #         points=data_dict['points'][:, 1:], ref_boxes=pred_dicts['pred_boxes'].detach(),
        #         ref_scores=pred_dicts['pred_scores'].detach(), ref_labels=pred_dicts['pred_labels'].detach(), confidence=0.5
        #     )
        
        # draw_scenes(vis,
        #         points=data_dict['points'][:, 1:], ref_boxes=None,
        #         ref_scores=None, ref_labels=None, confidence=0.5
        #     )
        

        vis.poll_events()
        vis.update_renderer()
        vis.clear_geometries()


## Open3d window is black screen-Open3d 黑屏

It‘s normal that the Open3d window initializes with black screen for the view is not initialized properly.

Open3d 刚打开是黑屏是正常现象，因为视角没有正常初始化。

Use `ctrl+C` to copy the following json and `ctrl+v` to paste it in the Open3d window.

用 `ctrl+C` 复制下面的 json 并使用`ctrl+v`粘贴到所打开的Open3d窗口中：

```json
{ 
	"class_name" : "ViewTrajectory",
	"interval" : 29,
	"is_loop" : false, 
	"trajectory" :  
	[
		{
			"boundingbox_max" : [ 69.118263244628906, 39.679920196533203, 16.415634155273438 ],
			"boundingbox_min" : [ -0.059999999999999998, -39.679874420166016, -6.9146575927734375 ],
			"field_of_view" : 60.0,
			"front" : [ -0.90307097537632919, 0.0017988087570628851, 0.42948757574567964 ],
			"lookat" : [ 34.529131622314452, 2.288818359375e-05, 4.75048828125 ],
			"up" : [ 0.42948904059539766, 0.0070563614983622357, 0.90304450154510629 ],
			"zoom" : 0.69999999999999996
		}
	],
	"version_major" : 1,
	"version_minor" : 0
}
```



(161070, 4)
(79241, 4)

In [ ]:
client_b.disconnect_client()


In [ ]:
server_a.close_server()